In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle
from keras.models import Model
import os
from keras.applications.vgg16 import VGG16
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from IPython.display import clear_output
import xgboost as xgb
import gc



In [ ]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 501 #Resize images (height  = X, width = Y)
SIZE_Y = 501


#Capture training image info as a list
train_images = []

dirname = './dataset/data'  
for fname in os.listdir(dirname):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)   
 
    img = cv2.resize(img, (SIZE_Y, SIZE_X))
    train_stack = np.stack((img,)*3, axis=-1)
    train_images.append(train_stack)
    #train_labels.append(label)
#Convert list to array for machine learning processing        
dataimages = np.array(train_images)


#Capture mask/label info as a list
train_masks = [] 
dirname = './dataset/labels'
for fname in os.listdir(dirname):
    img = cv2.imread(os.path.join(dirname, fname), cv2.IMREAD_GRAYSCALE)       
    img = cv2.resize(img, (SIZE_Y, SIZE_X))
    #mask_stack = np.stack((img,)*3, axis=-1)
    train_masks.append(img)
    #print(f"Unique vlaues in labels image {np.unique(img)}")
        #train_labels.append(label)
#Convert list to array for machine learning processing          
labelimages = np.array(train_masks)
X_train, X_test, y_train, y_test = train_test_split(dataimages, labelimages, test_size=0.95, random_state=42)


#Use customary x_train and y_train variables

y_train = np.expand_dims(y_train, axis=3) #May not be necessary.. leftover from previous code 
y_test = np.expand_dims(y_test, axis=3)

#Load VGG16 model wothout classifier/fully connected layers
#Load imagenet weights that we are going to use as feature generators
#Talked to the professor and as it seems the input shape makes sure that we can actually run pooling and kernal operations
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE_X, SIZE_Y, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0

#After the first 2 convolutional layers the image dimension changes. 
#So for easy comparison to Y (labels) let us only take first 2 conv layers
#and create a new model to extract features
#New model with only first 2 conv layers
VGG_model = Model(inputs=VGG_model.input, outputs=VGG_model.get_layer('block1_conv2').output)
VGG_model.summary()
features = VGG_model.predict(X_train)

In [ ]:
#Plot the extracted features
square = 8
ix=1
for _ in range(square):
    for _ in range(square):
        ax = plt.subplot(square, square, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(features[0,:,:,ix-1], cmap='gray')
        ix +=1
plt.show()

In [ ]:
#Reassign 'features' as X to make it easy to follow
X=features
X = X.reshape(-1, X.shape[3])  #Make it compatible for Random Forest and match Y labels
Y = y_train.reshape(-1)

#Combine X and Y into a dataframe to make it easy to drop all rows with Y values 0

dataset = pd.DataFrame(X)
dataset['Label'] = Y

In [ ]:
# This is so that we dont overwork our Ram
dataset = dataset.sample(frac=0.005, random_state=1)

#Redefine X and Y for Random Forest
X_for_training = dataset.drop(labels = ['Label'], axis=1)
X_for_training = X_for_training.values  #Convert to array
Y_for_training = dataset['Label']
Y_for_training = Y_for_training.values  #Convert to array
mapping = {0: 0, 128: 1, 255: 2}
# Vectorized mapping
Y_for_training = np.vectorize(mapping.get)(Y_for_training)

In [ ]:
model = xgb.XGBClassifier()
# Train the model on training data
model.fit(X_for_training, Y_for_training) 
#Save model for future use
filename = 'model_XG.sav'
pickle.dump(model, open(filename, 'wb'))
print("Model training complete and saved.")

In [ ]:


# Convert the data to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_for_training, dtype=tf.float32)
Y_train_tensor = tf.convert_to_tensor(Y_for_training, dtype=tf.int64)

# Create a TensorFlow Dataset and a DataLoader
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, Y_train_tensor))
train_loader = train_dataset.shuffle(buffer_size=10000).batch(32)

# Define the neural network architecture
class SimpleNN(tf.keras.Model):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = layers.Dense(128, activation='relu')  # Adjust input dimensions if needed
        self.fc2 = layers.Dense(3)  # Adjust output dimensions based on the number of classes

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Initialize the model
model = SimpleNN()

# Define loss function and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Metrics to track loss
train_loss = tf.keras.metrics.Mean(name='train_loss')

# Training function
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)

# Train the model
num_epochs = 250
best_loss = float('inf')

for epoch in range(num_epochs):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    
    for inputs, labels in train_loader:
        train_step(inputs, labels)

    current_loss = train_loss.result()
    if current_loss < best_loss:
        best_loss = current_loss
        model.save_weights('NN_Model_VGG_Features.h5')
        print(f'New best model saved with loss: {best_loss:.4f}')

    print(f'Epoch {epoch + 1}, Loss: {current_loss:.4f}')

print("Training complete.")

In [ ]:
#This is to print a picture using the models we have
filename = 'model_XG.sav'
loaded_model_XGB = pickle.load(open(filename, 'rb'))

In [ ]:
first_image_batch = np.expand_dims(X_test[0], axis=0)  # Reshape to (1, 501, 501, 3)
X_test_feature = VGG_model.predict(first_image_batch)
X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])
#These are not the exact possibilities that we want as it is the number of trees that hit as XGB is an esemble model. 
pred_probabilities = loaded_model_XGB.predict_proba(X_test_feature)
class_probabilities_df = pd.DataFrame(pred_probabilities, 
                                      columns=[f'Class_{i+1}_Prob' for i in range(pred_probabilities.shape[1])])

filtered_df = class_probabilities_df[(class_probabilities_df < 0.6).all(axis=1)]

prediction = loaded_model_XGB.predict(X_test_feature)
prediction_image = prediction.reshape((501, 501))

In [ ]:
num_pixels = 501 * 501
feature_matrix = X_test_feature.reshape((num_pixels, -1))

# Convert to TensorFlow tensor
feature_tensor = tf.convert_to_tensor(feature_matrix, dtype=tf.float32)

modelNN = SimpleNN()  # Replace with your actual model architecture
sample_input = tf.random.normal([1, 64])  # Replace 'input_shape' with the shape of your model input
modelNN(sample_input)
# Load the weights
modelNN.load_weights('NN_Model_VGG_Features.h5')
# Predict with the model
prediction = modelNN(feature_tensor)  # model is your loaded and compiled neural network model
prediction = tf.nn.softmax(prediction, axis=1)  # As classification dont need softmax we have it here to get probabilities and take the one with the highest argument in the end
predicted_classes = tf.argmax(prediction, axis=1).numpy()

# Reshape the predictions back into image format
prediction_image_NN = predicted_classes.reshape((501, 501))

In [ ]:
#Plot Predictions for the first image
original_image = np.squeeze(X_test[0])
labeled_images = np.squeeze(y_test[0])  # Removes the singleton dimension if it exists

print(original_image.shape)
print(labeled_images.shape)
# If X_test is a grayscale image with a shape like (n_samples, 501, 501), you might need to do:
# original_image = X_test[0].reshape((501, 501))

plt.figure(figsize=(15, 5))  # Increase the figure size for better visibility

# Plot the original image
plt.subplot(1, 4, 1)  # 1 row, 3 columns, first subplot
plt.imshow(original_image, cmap='gray')
plt.title('Original Image')

# Plot the labeled image
plt.subplot(1, 4, 2)  # 1 row, 3 columns, second subplot
plt.imshow(labeled_images, cmap='gray')  # You might not need cmap='gray' if it's a color label image
plt.title('Labeled Image')

# Plot the prediction image
plt.subplot(1, 4, 3)  # 1 row, 3 columns, third subplot
plt.imshow(prediction_image, cmap='gray')
plt.title('Prediction')

# Plot the prediction image
plt.subplot(1, 4, 4)  # 1 row, 3 columns, third subplot
plt.imshow(prediction_image_NN, cmap='gray')
plt.title('Prediction NN')

# Show the plot
plt.show()

In [64]:
 

def calculate_metrics(y_true, y_pred, class_values):
    y_pred_mapped = np.copy(y_pred)
    for class_index, class_value in enumerate(class_values):
        y_pred_mapped[y_pred == class_index] = class_value

    y_true_flatten = y_true.flatten()
    y_pred_flatten = y_pred_mapped.flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_true_flatten, y_pred_flatten)
    
    # For a multi-class problem, change 'binary' to 'macro' (unweighted average), 'micro' (global average), or 'weighted' (weighted average)
    precision = precision_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    recall = recall_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    f1 = f1_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    iou = jaccard_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    
    return accuracy, precision, recall, f1, iou

# Initialize lists to store metric scores for each image
accuracies = []
precisions = []
recalls = []
f1_scores = []
ious = []

for i in range(5):  # Iterate over each image in X_test len(X_test)
    # Prepare the image for prediction
    image_batch = np.expand_dims(X_test[i], axis=0)
    X_test_feature = VGG_model.predict(image_batch)
    X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])
    prediction = loaded_model_XGB.predict(X_test_feature)
    prediction_image = prediction.reshape((501, 501))
    
    # Convert prediction to the same format as ground truth, if necessary
    # Your conversion code here
    
    # Calculate metrics for the current prediction
    accuracy, precision, recall, f1, iou = calculate_metrics(y_test[i], prediction_image, [0, 128, 255])
    
    # Append metrics to the lists
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    ious.append(iou)
    
    # Clear memory periodically if needed
    if (i+1) % 10 == 0:  # Clear every 10 images, adjust based on your system's memory
        gc.collect()

# Calculate and print the mean of the metrics
mean_accuracy = np.mean(accuracies)
mean_precision = np.mean(precisions)
mean_recall = np.mean(recalls)
mean_f1_score = np.mean(f1_scores)
mean_iou = np.mean(ious)

print(f'Mean Accuracy: {mean_accuracy}')
print(f'Mean Precision: {mean_precision}')
print(f'Mean Recall: {mean_recall}')
print(f'Mean F1 Score: {mean_f1_score}')
print(f'Mean IoU: {mean_iou}')


1/1 [==============================] - 0s 37ms/step
Mean Accuracy: 0.9894159784223968
Mean Precision: 0.9895149139805218
Mean Recall: 0.9887047936989445
Mean F1 Score: 0.9891009052257885
Mean IoU: 0.9784568213564796


In [65]:
import numpy as np
import gc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score

def preprocess_image(image):
    """Preprocesses the image for model prediction."""
    return np.expand_dims(image, axis=0)



def predict_with_nn(nn_model, image):
    # Get features from VGG model
    feature_matrix = VGG_model.predict(image)
    feature_matrix = feature_matrix.reshape((501 * 501, -1))

    # Convert to TensorFlow tensor
    feature_tensor = tf.convert_to_tensor(feature_matrix, dtype=tf.float32)
    # Predict with the model
    prediction = modelNN(feature_tensor)  # model is your loaded and compiled neural network model
    prediction = tf.nn.softmax(prediction, axis=1)  # As classification dont need softmax we have it here to get probabilities and take the one with the highest argument in the end
    predicted_classes = tf.argmax(prediction, axis=1).numpy()

    # Reshape the predictions back into image format
    prediction_image_NN = predicted_classes.reshape((501, 501))
    return prediction_image_NN

def predict_with_xgb( xgb_model, image):
    feature_matrix = VGG_model.predict(image)
    feature_matrix = feature_matrix.reshape(-1, feature_matrix.shape[3])
    prediction = xgb_model.predict(feature_matrix)
    prediction_image = prediction.reshape((501, 501))
    return prediction_image
    
def calculate_metrics(y_true, y_pred, class_values):
    y_pred_mapped = np.copy(y_pred)
    for class_index, class_value in enumerate(class_values):
        y_pred_mapped[y_pred == class_index] = class_value

    y_true_flatten = y_true.flatten()
    y_pred_flatten = y_pred_mapped.flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_true_flatten, y_pred_flatten)
    
    # For a multi-class problem, change 'binary' to 'macro' (unweighted average), 'micro' (global average), or 'weighted' (weighted average)
    precision = precision_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    recall = recall_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    f1 = f1_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    iou = jaccard_score(y_true_flatten, y_pred_flatten, average='macro', labels=class_values)
    
    return accuracy, precision, recall, f1, iou

def compute_and_print_mean_metrics(metrics):
    """Computes and prints the mean of the given metrics."""
    mean_metrics = np.mean(metrics, axis=0)
    metric_names = ["Accuracy", "Precision", "Recall", "F1 Score", "IoU"]

    for name, mean_metric in zip(metric_names, mean_metrics):
        print(f'Mean {name}: {mean_metric}')

# Initialize lists to store metric scores for each image for both models
nn_metrics, xgb_metrics = [], []

for i in range(5):
    image = preprocess_image(X_test[i])

    # Predict with Neural Network and XGBoost models
    xgb_prediction = predict_with_xgb(loaded_model_XGB, image)

    nn_prediction = predict_with_nn(modelNN, image)

    # Convert prediction to the same format as ground truth, if necessary
    # Example: nn_prediction_converted = convert_format(nn_prediction)
    # Example: xgb_prediction_converted = convert_format(xgb_prediction)

    # Calculate metrics
    nn_metrics.append(calculate_metrics(y_test[i].flatten(), nn_prediction.flatten(),[0, 128, 255]))
    xgb_metrics.append(calculate_metrics(y_test[i].flatten(), xgb_prediction.flatten(),[0, 128, 255]))

    # Memory management
    if (i + 1) % 10 == 0:  # Clear memory every 10 iterations
        gc.collect()

# Print mean metrics for both models
print("Neural Network Model Metrics:")
compute_and_print_mean_metrics(nn_metrics)

print("\nXGBoost Model Metrics:")
compute_and_print_mean_metrics(xgb_metrics)



1/1 [==============================] - 0s 47ms/step
Neural Network Model Metrics:
Mean Accuracy: 0.9822295528703073
Mean Precision: 0.9815346536857545
Mean Recall: 0.9819457644918004
Mean F1 Score: 0.9817308463250198
Mean IoU: 0.9641560547648471

XGBoost Model Metrics:
Mean Accuracy: 0.9894159784223968
Mean Precision: 0.9895149139805218
Mean Recall: 0.9887047936989445
Mean F1 Score: 0.9891009052257885
Mean IoU: 0.9784568213564796
